In [280]:
#model input 형식

{'input_ids': tensor([[    0, 42891,    99,    32,   608,   452, 17487,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])}
{'input_ids': tensor([[   0,  118,  524,  205, 2156,  939,   95,  300,  160,  173,    8, 7428,
         2156,  939,   33,   80, 1315,  479,    2,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])}


In [58]:
import torch
import numpy as np
import pickle
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast, AutoTokenizer

In [59]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large')
model = RobertaForSequenceClassification.from_pretrained('../output/roberta-large/checkpoint-2200')
model.to(0)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [136]:
ct = 0
with open('../data/convai2/train_both_revised_no_cands.txt') as f:
    train_new_sen = []
    for i in f.readlines():
    
        if len(i.split('\t')) == 1:
            continue
        for j in i.split('\t'):
            while j[0].isnumeric():
                j = j[1:]
            j=j.strip()
            
            if j == '__SILENCE__':
                ct += 1
                continue
            if j == '':
                continue
            
                
            train_new_sen.append(j)
print('except silence and white num : ' , ct)

train_new_sen = list(set(train_new_sen))
print('final train data len: ', len(train_new_sen))

except silence and white num :  8939
final train data len:  124985


In [137]:
with open('../data/convai2/valid_both_revised_no_cands.txt') as f:
    val_new_sen = []
    for i in f.readlines():
    
        if len(i.split('\t')) == 1:
            continue
        for j in i.split('\t'):
            
            while j[0].isnumeric():
                j = j[1:]
            j=j.strip()
            val_new_sen.append(j)


val_new_sen = list(set(val_new_sen))
print('final val data len: ', len(val_new_sen))

final val data len:  15218


In [151]:
with open('../data/convai2/test_both_revised.txt') as f:
    test_new_sen = []
    ct = 0
    for i in f.readlines():
        if ct ==10 :
            break
            
        if len(i.split('\t')) == 1:
            continue
        for j in i.split('\n')[0].split('\t')[:2]:
            
            while j[0].isnumeric():
                j = j[1:]
            
            j=j.strip()
            test_new_sen.append(j)

print('test set len : ', len(test_new_sen))
test_new_sen = list(set(test_new_sen))
print('final test data len : ', len(test_new_sen))

test set len :  15024
final test data len :  14623


In [140]:
inputs_train = tokenizer(train_new_sen, truncation=True, padding=True)
inputs_valid = tokenizer(val_new_sen, truncation=True, padding=True)
inputs_test = tokenizer(test_new_sen, truncation=True, padding=True)

In [143]:
tr_dataloader = []
val_dataloader = []
tt_dataloader = []

for i in range(len(inputs_train['input_ids'])):
    tmp_dic = {}
    tmp_dic['input_ids']= torch.tensor([inputs_train['input_ids'][i]])
    tmp_dic['attention_mask']= torch.tensor([inputs_train['attention_mask'][i]])
    tr_dataloader.append(tmp_dic)

for i in range(len(inputs_valid['input_ids'])):
    tmp_dic = {}
    tmp_dic['input_ids']= torch.tensor([inputs_valid['input_ids'][i]])
    tmp_dic['attention_mask']= torch.tensor([inputs_valid['attention_mask'][i]])
    val_dataloader.append(tmp_dic)

for i in range(len(inputs_test['input_ids'])):
    tmp_dic = {}
    tmp_dic['input_ids']= torch.tensor([inputs_test['input_ids'][i]])
    tmp_dic['attention_mask']= torch.tensor([inputs_test['attention_mask'][i]])
    tt_dataloader.append(tmp_dic)

In [144]:
tt_dataloader

[{'input_ids': tensor([[    0,   118,   524,  3027,    11, 30081, 44347,   479,   939,   657,
            2480,  6353,    25,   157,   479,     2,     1,     1,     1,     1,
               1,     1,     1,     1,     1,     1,     1,     1]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0]])},
 {'input_ids': tensor([[   0, 9178,   32,   47,  608,   14,    8,  447,   15,   10,  478,    2,
              1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
              1,    1,    1,    1]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0]])},
 {'input_ids': tensor([[    0,  6025,    16,    45,  1528,  2156,    33,    47,   702,   143,
            1612, 17487,     2,     1,     1,     1,     1,     1,     1,     1,
               1,     1,     1,     1,     1,     1,     1,     1]]),
  'attention_mask': tensor([[1, 1, 1,

In [146]:
from fastprogress.fastprogress import master_bar, progress_bar
model.eval()

tr_labeled_data = []

id2label = model.config.id2label


for batch in progress_bar(tr_dataloader):

    tmp=[]  
    
    r_input_ids =batch['input_ids'].numpy()[0].tolist()
    tmp.append(tokenizer.decode(r_input_ids, skip_special_tokens=True))
    
    batch = tuple(batch[t].to(0) for t in batch)
    inputs = { 'input_ids' : batch[0], 'attention_mask' : batch[1]}
    outputs = model(**inputs)
    
    tmp.append(np.argmax(outputs[0].detach().cpu().numpy(), axis = 1)[0])
    tr_labeled_data.append(tmp)

In [147]:
val_labeled_data = []

for batch in progress_bar(val_dataloader):

    tmp=[]    
    
    r_input_ids =batch['input_ids'].numpy()[0].tolist()
    tmp.append(tokenizer.decode(r_input_ids, skip_special_tokens=True))
    
    batch = tuple(batch[t].to(0) for t in batch)
    inputs = { 'input_ids' : batch[0], 'attention_mask' : batch[1]}
    outputs = model(**inputs)
    
    tmp.append(np.argmax(outputs[0].detach().cpu().numpy(), axis = 1)[0])
    val_labeled_data.append(tmp)

In [148]:
tt_labeled_data = []

for batch in progress_bar(tt_dataloader):

    tmp=[]  
    
    r_input_ids =batch['input_ids'].numpy()[0].tolist()
    tmp.append(tokenizer.decode(r_input_ids, skip_special_tokens=True))
    
    batch = tuple(batch[t].to(0) for t in batch)
    inputs = { 'input_ids' : batch[0], 'attention_mask' : batch[1]}
    outputs = model(**inputs)
    
    tmp.append(np.argmax(outputs[0].detach().cpu().numpy(), axis = 1)[0])
    tt_labeled_data.append(tmp)

In [149]:
tr_labeled_data.sort(key= lambda x : x[1])
val_labeled_data.sort(key =lambda x : x[1])
tt_labeled_data.sort(key=lambda x : x[1])

In [154]:
check_pre_label = 0
idx = 0

for i in range(len(tr_labeled_data)):
    
    if check_pre_label != tr_labeled_data[i][1]:
        
        print('{} {}'.format(id2label[tr_labeled_data[i-1][1]], int(tr_labeled_data[i-1][2])+1))
        
        idx = 0
        check_pre_label = tr_labeled_data[i][1]
                
        tr_labeled_data[i].append(idx)
        idx += 1
        
    else:
        tr_labeled_data[i].append(idx)
        idx += 1
    
    if i == len(tr_labeled_data)-1:
        print('{} {}'.format(id2label[tr_labeled_data[i-1][1]],int(tr_labeled_data[i-1][2])+1))

trusting 1237
excited 7298
annoyed 3403
joyful 5672
ashamed 3003
sad 2347
devastated 913
nostalgic 10756
terrified 703
caring 9962
embarrassed 1854
anxious 2960
lonely 3367
confident 15261
jealous 3799
impressed 5704
faithful 2411
guilty 1333
disgusted 2248
content 10033
grateful 1501
surprised 7924
afraid 2252
sentimental 4130
anticipating 1471
furious 70
hopeful 6451
apprehensive 1234
proud 2120
angry 559
disappointed 565
prepared 2443


In [155]:
check_pre_label = 0
idx = 0

for i in range(len(val_labeled_data)):
    
    if check_pre_label != val_labeled_data[i][1]:
        
        print('{} {}'.format(id2label[val_labeled_data[i-1][1]],int(val_labeled_data[i-1][2])+1))
        
        idx = 0
        check_pre_label = val_labeled_data[i][1]
        
        val_labeled_data[i].append(idx)
        idx += 1
        
    else:
        val_labeled_data[i].append(idx)
        idx += 1
    
    if i == len(val_labeled_data)-1:
        print('{} {}'.format(id2label[val_labeled_data[i-1][1]],int(val_labeled_data[i-1][2])+1))

trusting 160
excited 868
annoyed 333
joyful 739
ashamed 326
sad 275
devastated 92
nostalgic 1360
terrified 144
caring 1311
embarrassed 203
anxious 294
lonely 421
confident 1820
jealous 457
impressed 713
faithful 336
guilty 159
disgusted 219
content 1099
grateful 212
surprised 952
afraid 328
sentimental 524
anticipating 137
furious 14
hopeful 845
apprehensive 139
proud 326
angry 69
disappointed 52
prepared 290


In [156]:
check_pre_label = 0
idx = 0

for i in range(len(tt_labeled_data)):
    
    if check_pre_label != tt_labeled_data[i][1]:
        
        print('{} {}'.format(id2label[tt_labeled_data[i-1][1]], int(tt_labeled_data[i-1][2])+1))
        
        idx = 0
        check_pre_label = tt_labeled_data[i][1]
        
        tt_labeled_data[i].append(idx)
        idx += 1
        
    else:
        tt_labeled_data[i].append(idx)
        idx += 1
    
    if i == len(tt_labeled_data)-1:
        print('{} {}'.format(id2label[tt_labeled_data[i-1][1]], int(tt_labeled_data[i-1][2])+1))

trusting 138
excited 770
annoyed 339
joyful 695
ashamed 320
sad 244
devastated 105
nostalgic 1242
terrified 128
caring 1377
embarrassed 179
anxious 300
lonely 479
confident 1724
jealous 422
impressed 619
faithful 331
guilty 124
disgusted 273
content 1101
grateful 189
surprised 972
afraid 295
sentimental 537
anticipating 139
furious 12
hopeful 774
apprehensive 145
proud 276
angry 57
disappointed 45
prepared 271


In [157]:
import jsonlines
from collections import OrderedDict
import json

with open("train.jsonl", "w", encoding="utf-8") as f:
    
    for i in range(len(tr_labeled_data)):
        my_data = OrderedDict()
        my_data["index"] = int(tr_labeled_data[i][2])
        my_data["X"] = tr_labeled_data[i][0]
        my_data["Y"] = tr_labeled_data[i][0]
        my_data["C"] = int(tr_labeled_data[i][1])

        json.dump(my_data, f, ensure_ascii=False) 
        f.write("\n")

In [158]:
import jsonlines
from collections import OrderedDict

with open("test.jsonl", "w", encoding="utf-8") as f:
    
    for i in range(len(tt_labeled_data)):
        my_data = OrderedDict()
        my_data["index"] = int(tt_labeled_data[i][2])
        my_data["X"] = tt_labeled_data[i][0]
        my_data["Y"] = tt_labeled_data[i][0]
        my_data["C"] = int(tt_labeled_data[i][1])

        json.dump(my_data, f, ensure_ascii=False) 
        f.write("\n")

In [159]:
import jsonlines
from collections import OrderedDict

with open("dev.jsonl", "w", encoding="utf-8") as f:
    
    for i in range(len(val_labeled_data)):
        my_data = OrderedDict()
        my_data["index"] = int(val_labeled_data[i][2])
        my_data["X"] = val_labeled_data[i][0]
        my_data["Y"] = val_labeled_data[i][0]
        my_data["C"] = int(val_labeled_data[i][1])

        json.dump(my_data, f, ensure_ascii=False) 
        f.write("\n")

Convai2 데이터셋 감정 라벨 분포

trainset 

'trusting' : 2606,
'excited' : 14339,
'annoyed' : 6543,
'joyful' : 16782,
'ashamed' : 5810,
'sad' : 4528,
'devastated'  : 1829, 
'nostalgic'  : 21459,
'terrified'  : 1350,
'caring'  : 20868,
'embarrassed'  : 3554, 
'anxious'  : 5723,
'lonely'  : 6604,
'confident'  : 31158, 
'jealous'  : 7353,
'impressed'  : 11041, 
'faithful'  : 4698
'guilty'  : 2595,
'disgusted'  : 4315, 
'content'  : 20843
'grateful'  : 2854
'surprised'  : 15795
'afraid'  : 4326
'sentimental'  : 8001
'anticipating'  : 2773
'furious'  : 131
'hopeful'  : 12699
'apprehensive'  : 2365
'proud' : 4113
'angry' : 1077
'disappointed' : 1084
'prepared' : 4721

validset 

trusting 개수 : 168
excited 개수 : 869
annoyed 개수 : 334
joyful 개수 : 932
ashamed 개수 : 326
sad 개수 : 275
devastated 개수 : 93
nostalgic 개수 : 1383
terrified 개수 : 144
caring 개수 : 1347
embarrassed 개수 : 203
anxious 개수 : 294
lonely 개수 : 424
confident 개수 : 1858
jealous 개수 : 457
impressed 개수 : 718
faithful 개수 : 336
guilty 개수 : 160
disgusted 개수 : 219
content 개수 : 1148
grateful 개수 : 215
surprised 개수 : 958
afraid 개수 : 328
sentimental 개수 : 524
anticipating 개수 : 137
furious 개수 : 14
hopeful 개수 : 858
apprehensive 개수 : 139
proud 개수 : 328
angry 개수 : 69
disappointed 개수 : 52
prepared 개수 : 292

testset 

trusting 개수 : 140
excited 개수 : 773
annoyed 개수 : 340
joyful 개수 : 893
ashamed 개수 : 320
sad 개수 : 246
devastated 개수 : 106
nostalgic 개수 : 1265
terrified 개수 : 128
caring 개수 : 1435
embarrassed 개수 : 180
anxious 개수 : 301
lonely 개수 : 479
confident 개수 : 1761
jealous 개수 : 422
impressed 개수 : 621
faithful 개수 : 333
guilty 개수 : 124
disgusted 개수 : 273
content 개수 : 1135
grateful 개수 : 189
surprised 개수 : 989
afraid 개수 : 296
sentimental 개수 : 539
anticipating 개수 : 139
furious 개수 : 12
hopeful 개수 : 790
apprehensive 개수 : 145
proud 개수 : 276
angry 개수 : 57
disappointed 개수 : 45
prepared 개수 : 271

중복 제거 감정 라벨 분포

train 

trusting 1237
excited 7298
annoyed 3403
joyful 5672
ashamed 3003
sad 2347
devastated 913
nostalgic 10756
terrified 703
caring 9962
embarrassed 1854
anxious 2960
lonely 3367
confident 15261
jealous 3799
impressed 5704
faithful 2411
guilty 1333
disgusted 2248
content 10033
grateful 1501
surprised 7924
afraid 2252
sentimental 4130
anticipating 1471
furious 70
hopeful 6451
apprehensive 1234
proud 2120
angry 559
disappointed 565
prepared 2444

dev

trusting 160
excited 868
annoyed 333
joyful 739
ashamed 326
sad 275
devastated 92
nostalgic 1360
terrified 144
caring 1311
embarrassed 203
anxious 294
lonely 421
confident 1820
jealous 457
impressed 713
faithful 336
guilty 159
disgusted 219
content 1099
grateful 212
surprised 952
afraid 328
sentimental 524
anticipating 137
furious 14
hopeful 845
apprehensive 139
proud 326
angry 69
disappointed 52
prepared 291

test

trusting 138
excited 770
annoyed 339
joyful 695
ashamed 320
sad 244
devastated 105
nostalgic 1242
terrified 128
caring 1377
embarrassed 179
anxious 300
lonely 479
confident 1724
jealous 422
impressed 619
faithful 331
guilty 124
disgusted 273
content 1101
grateful 189
surprised 972
afraid 295
sentimental 537
anticipating 139
furious 12
hopeful 774
apprehensive 145
proud 276
angry 57
disappointed 45
prepared 272